# Fluorescence decay

In [1]:
import pandas as pd
import numpy as np

import IMP
import IMP.bff

In [2]:
dt = 0.0141 # time resolution (bin width of decay histogram)

fns = {
    "lintable": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_whitelight.txt"),
    "irf": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_Prompt.txt"),
    "donor": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_Donor.txt"),
    "fret": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_FRET.txt")
}

data = dict()
for key in fns:
    df = pd.read_csv(fns[key], skiprows=6, sep='\t')
    data[key] = IMP.bff.DecayCurve(x=df['Chan'] * dt, y=df['Data'])

In [3]:
class CB(IMP.bff.CnNodeCallback):

    def __init__(self, *args, **kwargs):
        super(CB, self).__init__(*args, **kwargs)

    def run(self, inputs, outputs):
        pass

def init(obj, *args, **kwargs):
    super(obj.__class__, obj).__init__(*args, **kwargs)
    
    cls_kwargs = obj.cls_init
    obj.cb = CB()
    obj.set_callback(obj.cb)
    cl_name = obj.cls_name.split(".")[-1]
    cl = getattr(IMP.bff, cl_name)
    obj.cls = cl(**cls_kwargs)
    for k in obj.cls_in:
        p = IMP.bff.CnPort()

    # Method for CB class
    def _in2out(inputs, outputs):
        for k in obj.cls_in:
            setattr(obj.cls, k, inputs[k].value)
        for k in obj.cls_out:
            v = getattr(obj.cls, k)
            outputs[k].value = v
    
    # Make Ports of CnNode
    for k in obj.cls_in:
        v = cls_kwargs[k]
        p = IMP.bff.CnPort(v, is_output=False)
        obj.add_input_port(k, p)
    for k in obj.cls_out:
        v = cls_kwargs[k]
        p = IMP.bff.CnPort(v, is_output=True)
        obj.add_output_port(k, p)
    
    obj.cb.run = _in2out


In [4]:
settings = {
    "background_settings": {
        "start": 0, "stop": -1,
        "constant_offset": 2.,
        "pattern": None,
        "pattern_fraction": 0.0,
        "active": True
    },
    "pile_up_settings": {
        "active": False
    },
    "convolution_settings": {
        "start": 100, "stop": 3800,
        "instrument_response_function": data["irf"],
        "convolution_method": IMP.bff.DecayConvolution.FAST,
        "excitation_period": 100,
        "irf_shift_channels": 0.0,
        "irf_background_counts": 0.0,
    },
    "scale_settings": {
        "start": 0, "stop": -1,
        "constant_background": 0.0,
        "active": True,
        "blank_outside": True
    },
    "score_settings": {
        "score_type": "poisson",
        "start": 0, "stop": -1    
    }
}

In [5]:
attr_template = {
    "cls_init" : {},
    "cls_in": [],
    "cls_out": [],
    "cls_name": "IMP.bff.None",
    "__init__": init
}

cls_dict = {
    "CnLifetime": {
        "cls_init" : {
            'lifetime_spectrum': [0.012, 4.0, 2.2, 2.0],
            'abs_lifetime_spectrum': True,
            'use_amplitude_threshold': True,
            'amplitude_threshold': 0.0
        },
        "cls_in": [
            "amplitude_threshold",
            "lifetime_spectrum"
        ],
        "cls_out": [
            "lifetime_spectrum"
        ],
        "cls_name": "IMP.bff.DecayLifetimeHandler"
    },
    "CnDecay": {
        "cls_init" : {
            'x': [], 'y': []
        },
        "cls_in": ["x", "y"],
        "cls_out": ["x", "y"],
        "cls_name": "IMP.bff.DecayCurve"
    }
}

cls_ = dict()
for class_name in cls_dict:
    attr = {**attr_template, **cls_dict[class_name]}
    cls_[class_name] = type(class_name, (IMP.bff.CnNode, ), attr)


locals().update(cls_)

In [6]:
n_lt1 = CnLifetime()
n_decay = CnDecay()

In [7]:
n_decay.inputs['y'].value = 10 * [0.0]

In [8]:
n_decay.evaluate()

In [15]:
n_decay.outputs['x'].value

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [10]:
n_lt1.evaluate()

In [11]:
n_lt1.outputs["lifetime_spectrum"].value

array([0.012, 4.   , 2.2  , 2.   ])

In [12]:
n_decay.outputs["y"].value

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
n_lt2.cls_init

NameError: name 'n_lt2' is not defined